In [3]:
from elasticsearch import Elasticsearch

In [5]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","2_GVpd5-eAWZf10CGmxS"),
    ca_certs="/Users/sathishkumarchandran/elasticsearch-8.11.0/config/certs/http_ca.crt"
)
es.ping()

True

## Prepare the data

In [6]:
import pandas as pd

df = pd.read_csv("myntra_products_catalog.csv").loc[:499]
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [7]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           468
                                                                                   True             32
Name: count, dtype: int64

In [8]:
df.fillna("None", inplace=True)

## Convert the relevant field to Vector using BERT model

In [11]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [12]:
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))

In [13]:
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.027645895, -0.0026341379, -0.0035884182, 0...."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.02466072, -0.028755408, -0.020332498, 0.03..."
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,"[-0.046943244, 0.081828, 0.048335172, -0.00015..."
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,"[-0.015098789, -0.010285395, 0.0094873, -0.023..."
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,"[-0.017746538, 0.006209655, 0.021813951, 0.026..."


In [21]:
df["DescriptionVector"].iloc[0]

array([ 2.76458953e-02, -2.63413787e-03, -3.58841824e-03,  5.13588004e-02,
        3.09661310e-02,  1.40507957e-02,  7.27065932e-03,  3.13872620e-02,
       -6.23786971e-02, -3.82888573e-03,  3.15214135e-02,  7.55471811e-02,
        2.12644250e-03,  4.64892872e-02,  5.07448874e-02, -1.71941947e-02,
        1.22893630e-02, -1.95682757e-02, -9.64851603e-02,  3.52679566e-03,
       -1.83508862e-02,  2.58225389e-02, -3.12243979e-02,  5.46113178e-02,
        2.11241543e-02,  8.97172838e-03,  1.99165810e-02,  2.25223228e-02,
        5.33200195e-03, -5.24596833e-02,  2.56654937e-02, -2.64644492e-02,
        1.60460006e-02, -2.47585364e-02,  1.45433091e-06, -5.99340610e-02,
       -4.21298407e-02, -5.82049973e-03, -1.85585842e-02, -6.88412488e-02,
        4.52963151e-02,  7.06153177e-03,  3.00954878e-02, -7.18991458e-02,
        2.67853476e-02, -6.78085536e-02,  1.52005441e-02,  7.82435238e-02,
       -3.86550501e-02, -1.53425662e-02,  3.39756869e-02,  1.37783047e-02,
       -1.47240870e-02, -

In [22]:
len(df["DescriptionVector"].iloc[0])

768

In [14]:
es.ping()

True

## Create new index in ElasticSearch!

In [15]:
from indexMapping import indexMapping

es.indices.create(index="all_products", mappings=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products'})

## Ingest the data into index

In [16]:
record_list = df.to_dict("records")

In [17]:
for record in record_list:
    try:
        es.index(index="all_products", document=record, id=record["ProductID"])
    except Exception as e:
        print(e)

In [18]:
es.count(index="all_products")

ObjectApiResponse({'count': 481, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Search the data

In [19]:
input_keyword = "Blue Shoes"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500, 
}

res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])
res["hits"]["hits"]

/var/folders/cf/1n82cp8j5477wf_t828mwrlh0000gn/T/ipykernel_24183/2784084207.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])


[{'_index': 'all_products',
  '_id': '10018013',
  '_score': 0.6142942,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}},
 {'_index': 'all_products',
  '_id': '10005997',
  '_score': 0.5837572,
  '_source': {'ProductName': 'ID Men Navy Blue Solid Leather Mid-Top Sneakers',
   'Description': 'A pair of round-toe navy blue sneakers, has mid-top styling, lace-up detailLeather upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}}]